In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.externals import joblib
from collections import Counter

In [2]:
data = joblib.load('data.pkl')

In [3]:
data_dir = 'bbdc_2019_Bewegungsdaten/'

In [5]:
data.keys()

dict_keys(['features', 'lens', 'labels'])

In [7]:
def standard_scale(data, lens):
    features = data['features']
    features = np.concatenate(features, axis=0)
    features = StandardScaler().fit_transform(features)
    ret = []
    l = 0
    for t in lens:
        ret.append(features[l: l + t])
        l += t
    features = np.array(ret)
    data['features'] = features

In [11]:
def get_split(data, split):
    if split == 'train':
        return data['features'][: data['lens']['train']]
    elif split == 'valid':
        return data['features'][data['lens']['train']: data['lens']['train'] + data['lens']['valid']]
    elif split == 'test':
        return data['features'][data['lens']['train'] + data['lens']['valid']:]

In [8]:
def generate_batch_idx(n, batch_size, randomise=False):
    idx = np.arange(0, n)
    if randomise:
        np.random.shuffle(idx)
    for batch_idx in np.arange(0, n, batch_size):
        yield idx[batch_idx:batch_idx+batch_size]

In [ ]:
def generate_batches(data, split, batch_size,
                     randomise=False):
    features = get_split(data, split)
    labels = data['labels'][split]
    lens = data['lens'][split]
    for batch_idx in generate_batch_idx(n, batch_size, randomise):
        return features[batch_idx], labels[batch_idx], lens[batch_idx]

dict_keys(['features', 'lens', 'train_len', 'valid_len', 'test'])